In [2]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import os

# 读取shapefile
shapefile = "shp_path.shp"
gdf = gpd.read_file(shapefile)

# 读取大图tif文件
tif_file = "tif_path.tif"

# 设置输出文件夹
output_folder = "output_dir_path"
os.makedirs(output_folder, exist_ok=True)

with rasterio.open(tif_file) as src:
    for idx, row in gdf.iterrows():
        # 获取框的几何信息
        geometry = [row['geometry']]
        
        # 跳过无效的几何形状
        if geometry[0] is None:
            continue
        
        # 裁剪大图
        try:
            out_image, out_transform = mask(src, geometry, crop=True)
        except Exception as e:
            print(f"裁剪第 {idx} 个框时出错: {e}")
            continue
        
        # 创建子文件夹
        sub_folder = os.path.join(output_folder, f"output_{idx}")
        os.makedirs(sub_folder, exist_ok=True)
        
        # 保存裁剪后的图像
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        output_tif = os.path.join(sub_folder, f"output_image_{idx}.tif")
        with rasterio.open(output_tif, "w", **out_meta) as dest:
            dest.write(out_image)
        
        # 为裁剪后的图像生成新的shapefile
        new_gdf = gpd.GeoDataFrame({'geometry': geometry}, crs=gdf.crs)
        output_shapefile = os.path.join(sub_folder, f"output_shape_{idx}.shp")
        new_gdf.to_file(output_shapefile)

print("裁剪和shapefile生成完成")

E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attri

裁剪和shapefile生成完成
